In [1]:
# !pip3 install -r requirements.txt
# !pip3 install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [2]:
import pandas as pd
import streamlit
import altair as alt
from vega_datasets import data
import math

In [3]:
# Load in the datasets
fifa = pd.read_csv('fifa19.csv')
gps = pd.read_csv('worldcities.csv')

In [4]:
# Add new columns in FIFA 19 and GPS datasets to match player's countries of origin (nationalities)
fifa['Nationality Country'] = fifa['Nationality'].copy()
gps['Nationality Country'] = gps['country'].copy()

In [5]:
# Count of nationalities that we don't have country gps data for
nationalities = fifa['Nationality Country'].unique()
countries = gps['Nationality Country'].unique()
print(len(set(nationalities)-set(countries)))

26


In [6]:
# Replace country names in GPS dataset based on geographically equivalent but differently named country in FIFA 19 dataset
gps['Nationality Country'] = gps['Nationality Country'].replace('Congo (Kinshasa)', 'DR Congo')
gps['Nationality Country'] = gps['Nationality Country'].replace('Congo (Brazzaville)', 'Congo')
gps['Nationality Country'] = gps['Nationality Country'].replace('Gambia, The', 'Gambia')
gps['Nationality Country'] = gps['Nationality Country'].replace('Sao Tome And Principe', 'São Tomé & Príncipe')
gps['Nationality Country'] = gps['Nationality Country'].replace('Antigua And Barbuda', 'Antigua & Barbuda')
gps['Nationality Country'] = gps['Nationality Country'].replace('Czechia', 'Czech Republic')
gps['Nationality Country'] = gps['Nationality Country'].replace('Korea, South', 'Korea Republic')
gps['Nationality Country'] = gps['Nationality Country'].replace('Korea, North', 'Korea DPR')
gps['Nationality Country'] = gps['Nationality Country'].replace('China', 'China PR')
gps['Nationality Country'] = gps['Nationality Country'].replace('Cabo Verde', 'Cape Verde')
gps['Nationality Country'] = gps['Nationality Country'].replace('Ireland', 'Republic of Ireland')
gps['Nationality Country'] = gps['Nationality Country'].replace('Saint Kitts And Nevis', 'Saint Kitts & Nevis')
gps['Nationality Country'] = gps['Nationality Country'].replace('Trinidad And Tobago', 'Trinidad & Tobago')
gps['Nationality Country'] = gps['Nationality Country'].replace('Bosnia And Herzegovina', 'Bosnia & Herzegovina')

In [7]:
# Replace the 4 countries that make up the UK with 'United Kingdom' (FIFA => GPS)
fifa['Nationality Country'] = fifa['Nationality Country'].replace('England', 'United Kingdom')
fifa['Nationality Country'] = fifa['Nationality Country'].replace('Scotland', 'United Kingdom')
fifa['Nationality Country'] = fifa['Nationality Country'].replace('Wales', 'United Kingdom')
fifa['Nationality Country'] = fifa['Nationality Country'].replace('Northern Ireland', 'United Kingdom')

In [8]:
# Replace country names in FIFA 19 dataset based on geographically equivalent but differently named country in GPS dataset
fifa['Nationality Country'] = fifa['Nationality Country'].replace('FYR Macedonia', 'Macedonia')
fifa['Nationality Country'] = fifa['Nationality Country'].replace('Central African Rep.', 'Central African Republic')
fifa['Nationality Country'] = fifa['Nationality Country'].replace('Guinea Bissau', 'Guinea-Bissau')
fifa['Nationality Country'] = fifa['Nationality Country'].replace('Curacao', 'Curaçao')
fifa['Nationality Country'] = fifa['Nationality Country'].replace('Bosnia Herzegovina', 'Bosnia & Herzegovina')
fifa['Nationality Country'] = fifa['Nationality Country'].replace('St Kitts Nevis', 'Saint Kitts & Nevis')
fifa['Nationality Country'] = fifa['Nationality Country'].replace('St Lucia', 'Saint Lucia')
fifa['Nationality Country'] = fifa['Nationality Country'].replace('Ivory Coast', 'Côte D’Ivoire')

In [9]:
# Nationalities that we don't have country gps data for
nationalities = fifa['Nationality Country'].unique()
countries = gps['Nationality Country'].unique()
x = set(nationalities)-set(countries)
x

{'Montserrat', 'Palestine'}

In [10]:
# Get average latitude and longitude for each country in GPS dataset
gps_avg = gps.groupby('Nationality Country').agg({'lat': ['mean'], 'lng': ['mean']}).reset_index()
gps_avg = pd.DataFrame({'Nationality Country': gps_avg['Nationality Country'],
                        'Lat': gps_avg['lat']['mean'],
                        'Lng': gps_avg['lng']['mean']})

In [11]:
# Manually add GPS info for Montserrat and Palestine based on Google search result 
gps_avg = gps_avg.append({'Nationality Country': 'Montserrat', 'Lat': 16.7425, 'Lng': -62.1874}, ignore_index=True)
gps_avg = gps_avg.append({'Nationality Country': 'Palestine', 'Lat': 31.9522, 'Lng': 35.2332}, ignore_index=True)

In [12]:
# Save updated dfs to csv files
fifa.to_csv('clean_fifa.csv')
gps_avg.to_csv('clean_worldcities.csv')

In [13]:
# Merge fifa and gps dfs
fifa_and_gps = fifa.merge(gps_avg, how='left', left_on='Nationality Country', right_on='Nationality Country')

In [14]:
# Extract numeric value from cost string
def convert_amount(cost):
    if type(cost) is not str:  # handle nan cases
        return cost
    factor = 1
    factor = 1000 if cost[-1] == 'K' else factor
    factor = 1000000 if cost[-1] == 'M' else factor
    amount = float(cost[1:-1])*factor if factor > 1 else float(cost[1:])
    return amount

In [15]:
# Extract value from cost strings (units in €)
fifa_and_gps['Wage'] = fifa_and_gps['Wage'].apply(convert_amount)
fifa_and_gps['Value'] = fifa_and_gps['Value'].apply(convert_amount)
fifa_and_gps['Release Clause'] = fifa_and_gps['Release Clause'].apply(convert_amount)

In [16]:
# Get average age, overall rating, and potential rating for each country of player origin
country_avg = fifa_and_gps.groupby('Nationality Country').agg({'Age': ['mean', 'min', 'max'],
                                                               'Overall': ['mean', 'min', 'max'],
                                                               'Potential': ['mean', 'min', 'max'],
                                                               'International Reputation': ['mean', 'min', 'max'],
                                                               'Wage': ['mean', 'min', 'max'],
                                                               'Value': ['mean', 'min', 'max'],
                                                               'Release Clause': ['mean', 'min', 'max'],
                                                               'Lat': ['mean'],
                                                               'Lng': ['mean']
                                                              }).reset_index()
country_avg.columns = ['_'.join(x) if x[0] not in ['Nationality Country', 'Lat', 'Lng'] else x[0] for x in country_avg.columns.ravel()]
country_avg.to_csv('clean_fifa_country_avg.csv')
country_avg

,Nationality Country,Age_mean,Age_min,Age_max,Overall_mean,Overall_min,Overall_max,Potential_mean,Potential_min,Potential_max,...,Wage_min,Wage_max,Value_mean,Value_min,Value_max,Release Clause_mean,Release Clause_min,Release Clause_max,Lat,Lng
0,Afghanistan,22.500000,20,26,61.000000,59,63,67.750000,64,71,...,1000.0,2000.0,3.425000e+05,220000.0,450000.0,6.172500e+05,374000.0,878000.0,34.755439,67.488482
1,Albania,24.025000,18,34,65.925000,52,81,71.700000,62,86,...,1000.0,58000.0,1.920250e+06,80000.0,20000000.0,3.621784e+06,116000.0,35500000.0,41.137240,19.950260
2,Algeria,27.050000,19,35,70.633333,58,85,72.983333,62,85,...,1000.0,205000.0,4.670917e+06,160000.0,40500000.0,8.768946e+06,259000.0,78000000.0,33.199642,3.345492
3,Andorra,28.000000,28,28,62.000000,62,62,64.000000,64,64,...,1000.0,1000.0,2.900000e+05,290000.0,290000.0,3.840000e+05,384000.0,384000.0,42.522429,1.534929
4,Angola,25.866667,19,34,67.600000,60,78,71.533333,64,82,...,1000.0,44000.0,2.018333e+06,325000.0,10000000.0,3.475846e+06,463000.0,17800000.0,-11.636753,16.219473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,Uruguay,26.610738,18,37,70.429530,54,91,73.382550,58,91,...,0.0,455000.0,4.811074e+06,0.0,80000000.0,9.379158e+06,100000.0,164000000.0,-33.622469,-56.018887
157,Uzbekistan,29.500000,29,30,67.500000,60,75,67.500000,60,75,...,1000.0,17000.0,3.100000e+06,200000.0,6000000.0,6.725000e+06,250000.0,13200000.0,40.692264,66.230054
158,Venezuela,24.313433,18,34,67.268657,53,81,73.149254,55,84,...,0.0,42000.0,2.237313e+06,0.0,22500000.0,4.093058e+06,108000.0,35400000.0,9.539313,-67.539629
159,Zambia,22.222222,18,29,65.222222,56,73,73.777778,66,83,...,1000.0,7000.0,1.129444e+06,190000.0,3500000.0,2.174000e+06,356000.0,6600000.0,-13.342017,27.707497


In [17]:
# Create world map
# source: https://altair-viz.github.io/gallery/index.html#maps

# Data generators for the background
sphere = alt.sphere()
graticule = alt.graticule()

# Source of land data
source = alt.topo_feature(data.world_110m.url, 'countries')

# Layering and configuring the components
background = alt.layer(
    alt.Chart(sphere).mark_geoshape(fill='lightblue'),
    alt.Chart(graticule).mark_geoshape(stroke='white', strokeWidth=0.2),
    alt.Chart(source).mark_geoshape(fill='lightgray', stroke='black')
).project(
    type='equirectangular'
).properties(width=800, height=400).configure_view(stroke=None)

hover = alt.selection(type='single', on='mouseover', nearest=True, fields=['Lat', 'Lng'])

base = alt.Chart(country_avg).encode(
    longitude='Lng:Q',
    latitude='Lat:Q',
    tooltip=['Nationality Country']
)

points = base.mark_point().encode(
    color=alt.condition(~hover, alt.value('black'), alt.value('red')),
    size=alt.condition(~hover, alt.value(30), alt.value(100))
).add_selection(hover)

background + points

alt.LayerChart(...)